<h1 style="color:#1f77b4; text-align:left; font-size:40px;">
    Deploy
</h1>

<h3 style="color:#555; text-align:left;">
    API tranformation?, integration with company pipeline?, Monitoring Dashboards other..............
</h3>


<h2 style="color:#1f77b4; border-bottom: 3px solid #1f77b4; padding-bottom:4px;">
</h2>

## **4.4: Ordinal encoding and create dummy variables**
### **4.4.1: Creating a function to avoid data leakage when doing one hot enconding**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

def encode_data_kfold(X, y, to_encode_cols, skf):
    """
    K-Fold preprocessing:
      - OneHotEncoding nas colunas 'to_encode_cols'
      - Todas as outras colunas são tratadas como numéricas/contínuas (mantidas como estão)
    Returns uma lista de folds, cada um com X_train, X_val, y_train, y_val e o encoder.
    """

    folds = []

    # As restantes colunas do DataFrame
    remaining_cols = [col for col in X.columns if col not in to_encode_cols]

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\nFOLD {fold + 1}")
        print("-" * 30)

        # 1) Split dos dados
        X_train = X.iloc[train_idx].copy()
        X_val   = X.iloc[val_idx].copy()
        y_train = y.iloc[train_idx]
        y_val   = y.iloc[val_idx]

        # 2) One-Hot Encoder (fit somente no treino)
        encoder = OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False)
        encoder.fit(X_train[to_encode_cols])

        # 3) Transformações
        X_train_cat = encoder.transform(X_train[to_encode_cols])
        X_val_cat   = encoder.transform(X_val[to_encode_cols])

        # 4) Novos nomes das colunas
        new_cols = encoder.get_feature_names_out(to_encode_cols)

        # 5) DataFrames das colunas codificadas
        X_train_cat = pd.DataFrame(X_train_cat, columns=new_cols, index=X_train.index)
        X_val_cat   = pd.DataFrame(X_val_cat,   columns=new_cols, index=X_val.index)

        # 6) Colunas restantes (não codificadas)
        X_train_remaining = X_train[remaining_cols]
        X_val_remaining   = X_val[remaining_cols]

        # 7) Data final do fold
        X_train_final = pd.concat([X_train_remaining, X_train_cat], axis=1)
        X_val_final   = pd.concat([X_val_remaining,   X_val_cat],   axis=1)

        print("X_train_final shape:", X_train_final.shape)
        print("X_val_final shape:",   X_val_final.shape)

        # 8) Guardar tudo
        folds.append({
            "fold": fold + 1,
            "X_train": X_train_final,
            "X_val": X_val_final,
            "y_train": y_train,
            "y_val": y_val,
            "encoder": encoder
        })

    return folds


In [ ]:
categorical_cols=["svc_internet_type","svc_contract","svc_payment_method"]

folds_data = encode_data_kfold(X, y, categorical_cols, skf)


FOLD 1
------------------------------
X_train_final shape: (5634, 62)
X_val_final shape: (1409, 62)

FOLD 2
------------------------------
X_train_final shape: (5634, 62)
X_val_final shape: (1409, 62)

FOLD 3
------------------------------
X_train_final shape: (5634, 62)
X_val_final shape: (1409, 62)

FOLD 4
------------------------------
X_train_final shape: (5635, 62)
X_val_final shape: (1408, 62)

FOLD 5
------------------------------
X_train_final shape: (5635, 62)
X_val_final shape: (1408, 62)


In [ ]:
# obter o primeiro fold
fold1 = folds_data[0]

# Ver X_train_final
print(fold1["X_train"].head())

# Ver X_val_final
print(fold1["X_val"].head())

# Ver y_train e y_val
print(fold1["y_train"].head())
print(fold1["y_val"].head())



   demo_count demo_gender  demo_age  demo_under_30  demo_senior_citizen  \
0           1        Male        78              0                    1   
1           1      Female        74              0                    1   
2           1        Male        71              0                    1   
4           1      Female        80              0                    1   
5           1      Female        72              0                    1   

   demo_married  demo_dependents  demo_number_of_dependents  loc_count  \
0             0                0                          0          1   
1             1                1                          1          1   
2             0                1                          3          1   
4             1                1                          1          1   
5             0                1                          1          1   

     loc_country  ... zip_population Is_woman  Has_offer  \
0  United States  ...          68701        

# 4: ??

codigo para criar report

In [ ]:
# 2) VIF sem statsmodels
X_enc_vif, X_num_vif, vif_table = multicollinearity_vif_manual(
    X=X,
    binary_cols=binary_cols,
    continuous_cols=continuous_cols,
    ordinal_cols=ordinal_cols,
    object_cols=object_cols
)




          CHECK DE MULTICOLINEARIDADE - VIF (Variance Inflation Factor, sem statsmodels)

VIF por feature (ordenado do maior para o menor):

                                  feature        VIF
25        svc_total_long_distance_charges        inf
24                      svc_total_refunds        inf
23                      svc_total_charges        inf
26                      svc_total_revenue        inf
33           svc_total_extra_data_charges        inf
22                     svc_monthly_charge  13.513482
4                   svc_referred_a_friend  11.326714
2                            demo_married  10.626800
7                    svc_internet_service  10.243219
19                           loc_latitude   9.353867
31                   svc_tenure_in_months   7.584087
15                     svc_unlimited_data   5.887628
13                   svc_streaming_movies   5.578747
29                           loc_zip_code   5.084557
3                         demo_dependents   5.010564
28        

# REPORT

In [ ]:
def create_feature_report_structure(X, 
                                    binary_cols,
                                    continuous_cols,
                                    ordinal_cols,
                                    object_cols):
    
    import pandas as pd

    rows = []

    for col in binary_cols:
        rows.append([col, "binary"])

    for col in continuous_cols:
        rows.append([col, "continuous"])

    for col in ordinal_cols:
        rows.append([col, "ordinal"])

    for col in object_cols:
        rows.append([col, "categorical"])

    # Convert to DataFrame
    report = pd.DataFrame(rows, columns=["feature", "type"])

    # Reset index
    report.set_index("feature", inplace=True)

    return report


In [ ]:
report = create_feature_report_structure(
    X, binary_cols, continuous_cols, ordinal_cols, object_cols
)


In [ ]:
def add_variance_to_report(report, X):
    variances = X.var(numeric_only=True)
    report["variance"] = report.index.map(variances)
    return report


In [ ]:
from sklearn.feature_selection import chi2

def add_chisquare_to_report(report, X_encoded, y):
    # Seleccionar apenas as features aplicáveis
    applicable_cols = report[report["type"].isin(["binary","categorical"])].index
    
    X_sub = X_encoded[applicable_cols]

    # Calcular Chi-square
    chi_scores, p_values = chi2(X_sub, y)

    report.loc[applicable_cols, "chi2_score"] = chi_scores
    report.loc[applicable_cols, "chi2_pvalue"] = p_values

    return report


In [ ]:
def add_spearman_to_report(report, X, y):
    import scipy.stats as ss

    spearman_scores = {}
    
    for col in report.index:
        if report.loc[col,"type"] in ["continuous","ordinal"]:
            rho, p = ss.spearmanr(X[col], y)
            spearman_scores[col] = rho
        else:
            spearman_scores[col] = None

    report["spearman"] = report.index.map(spearman_scores)
    return report


In [ ]:
def add_mi_to_report(report, mi_table):
    report["MI"] = report.index.map(mi_table["mean_MI"].to_dict())
    return report


In [ ]:
def add_vif_to_report(report, vif_df):
    report["VIF"] = report.index.map(vif_df.set_index("feature")["VIF"].to_dict())
    return report


In [ ]:
def add_high_corr_pairs_to_report(report, high_corr_pairs):
    # Construir mapa col -> lista de features correlacionadas
    corr_map = {col: [] for col in report.index}

    for col1, col2, corrval in high_corr_pairs:
        if col1 in corr_map:
            corr_map[col1].append(col2)
        if col2 in corr_map:
            corr_map[col2].append(col1)

    report["high_corr_with"] = report.index.map(corr_map)
    return report


In [ ]:
def add_recommendation(report):

    rec = []

    for feature, row in report.iterrows():

        reasons = []

        if row.get("variance", 1) < 0.001:
            reasons.append("low variance")

        if row.get("MI", 999) < 0.01:
            reasons.append("low MI")

        if row.get("VIF", 0) > 10:
            reasons.append("high VIF")

        if row.get("high_corr_with"):
            if len(row["high_corr_with"]) > 0:
                reasons.append("high correlation")

        if row.get("chi2_pvalue", 0) > 0.05:
            reasons.append("weak chi-square")

        if len(reasons) == 0:
            rec.append("KEEP")
        else:
            rec.append("DROP: " + ", ".join(reasons))

    report["recommendation"] = rec
    return report


In [ ]:
def build_feature_selection_report(
    X,
    y,
    binary_cols,
    continuous_cols,
    ordinal_cols,
    object_cols,
    mi_table,
    vif_df,
    high_corr_pairs
):
    # Base
    report = create_feature_report_structure(X, binary_cols, continuous_cols, ordinal_cols, object_cols)

    # Variance
    report = add_variance_to_report(report, X)

    # Spearman
    report = add_spearman_to_report(report, X, y)

    # MI
    report = add_mi_to_report(report, mi_table)

    # VIF
    report = add_vif_to_report(report, vif_df)

    # Correlação entre features
    report = add_high_corr_pairs_to_report(report, high_corr_pairs)

    # Recomendação automática
    report = add_recommendation(report)

    return report


In [ ]:
report = build_feature_selection_report(
    X=X,
    y=y,
    binary_cols=binary_cols,
    continuous_cols=continuous_cols,
    ordinal_cols=ordinal_cols,
    object_cols=object_cols,
    mi_table=mi_table,
    vif_df=vif_table,
    high_corr_pairs=high_corr_pairs
)

report


,type,variance,spearman,MI,VIF,high_corr_with,recommendation
feature,,,,,,,
demo_under_30,binary,1.593740e-01,NaN,0.004139,2.851729,[],DROP: low MI
demo_senior_citizen,binary,1.358745e-01,NaN,0.016089,2.735039,[],KEEP
demo_married,binary,2.497476e-01,NaN,0.011723,10.626800,[svc_referred_a_friend],"DROP: high VIF, high correlation"
demo_dependents,binary,1.776693e-01,NaN,0.036187,5.010564,[demo_number_of_dependents],DROP: high correlation
svc_referred_a_friend,binary,2.482269e-01,NaN,0.013101,11.326714,[demo_married],"DROP: high VIF, high correlation"
svc_phone_service,binary,8.746938e-02,NaN,0.005402,2.661023,[],DROP: low MI
svc_multiple_lines,binary,2.439252e-01,NaN,0.004492,1.604067,[],DROP: low MI
svc_internet_service,binary,1.697477e-01,NaN,0.034546,10.243219,[],DROP: high VIF
svc_online_security,binary,2.045183e-01,NaN,0.015261,1.387523,[],KEEP
